<a href="https://colab.research.google.com/github/mskansara/multivariate_lstm/blob/master/Drought.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from numpy import array
import pandas as pd
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from keras.optimizers import Adam

NAME='DroughtPrediction'

#Get dataset
dataset = pd.read_csv('RoughSPI.csv')

dataset.index = dataset['Time']
dataset = dataset.drop(['Time'], axis=1)
features = dataset.drop(['SPI','Location'], axis=1)
output = dataset.drop(['Location','Rainfall','Average temp','Evapotranspiration'], axis=1)



#Removing hyphens from dates
# features['Time'] = features['Time'].replace('-','',regex=True)
# dataset.index.name = 'Time'
#Assigning numbers to location
#0-Amravati, 1-Akola, 2-Bhandara, 3-Beed, 4-Aurangabad
# features['Location'] = features['Location'].replace('Amravati',0,regex=True)
# features['Location'] = features['Location'].replace('Akola',1,regex=True)
# features['Location'] = features['Location'].replace('Bhandara',2,regex=True)
# features['Location'] = features['Location'].replace('Beed',3,regex=True)
# features['Location'] = features['Location'].replace('Aurangabad',4,regex=True)

#Normalize the data
# features['Rainfall'] = (features['Rainfall']-min(features['Rainfall'])) / (max(features['Rainfall'])-min(features['Rainfall']))
# features['Average temp'] = (features['Average temp']-min(features['Average temp'])) / (max(features['Average temp'])-min(features['Average temp']))
# features['Evapotranspiration'] = (features['Evapotranspiration']-min(features['Evapotranspiration'])) / (max(features['Evapotranspiration'])-min(features['Evapotranspiration']))

#Standardize the data
feature_columns = features.columns
output_columns = output.columns

scaler = preprocessing.StandardScaler()
features = scaler.fit_transform(features)
features = pd.DataFrame(features,columns=feature_columns)

#Removing the headers from features and output
new_header = features.iloc[0] #grab the first row for the header
features = features[1:] #take the data less the header row
features.columns = new_header

new_header = output.iloc[0] #grab the first row for the header
output = output[1:] #take the data less the header row
output.columns = new_header

#Splitting features and output
X_train, X_test, Y_train, Y_test = train_test_split(features, output, test_size=0.20)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

#Create a model
model = Sequential()

#Stacked LSTM
model.add(LSTM(128, input_shape=(1,3),activation='relu', return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(64,activation='relu', return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
# model.add(Dense(32,activation='relu'))
model.add(Dense(1, activation='relu'))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

optimizer = Adam(lr=1e-5, decay=1e-6)

model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])
# model.summary()

# fit model
history = model.fit(X_train,Y_train,
                    epochs=100, 
                    callbacks=[tensorboard],
                    validation_data = (X_test,Y_test))

trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

# Score model
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

#Calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(Y_train[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(Y_test[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


Train on 1007 samples, validate on 252 samples
Epoch 1/100
1007/1007 [==============================] - 15s 15ms/step - loss: 0.8223 - acc: 0.0794 - val_loss: 0.8859 - val_acc: 0.0675
Epoch 2/100
1007/1007 [==============================] - 0s 469us/step - loss: 0.8223 - acc: 0.0794 - val_loss: 0.8859 - val_acc: 0.0675
Epoch 3/100
1007/1007 [==============================] - 0s 487us/step - loss: 0.8223 - acc: 0.0794 - val_loss: 0.8859 - val_acc: 0.0675
Epoch 4/100
1007/1007 [==============================] - 0s 469us/step - loss: 0.8223 - acc: 0.0794 - val_loss: 0.8859 - val_acc: 0.0675
Epoch 5/100
1007/1007 [==============================] - 1s 594us/step - loss: 0.8223 - acc: 0.0794 - val_loss: 0.8859 - val_acc: 0.0675
Epoch 6/100
1007/1007 [==============================] - 1s 587us/step - loss: 0.8223 - acc: 0.0794 - val_loss: 0.8859 - val_acc: 0.0675
Epoch 7/100
1007/1007 [==============================] - 1s 565us/step - loss: 0.8223 - acc: 0.0794 - val_loss: 0.8859 - val_acc: 0